In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")
train_df =  pd.read_csv('C:/Users/madhu/Desktop/train.csv')
test_df  =pd.read_csv('C:/Users/madhu/Desktop/test.csv')
sub_df = pd.read_csv('C:/Users/madhu/Desktop/sample_submission.csv')
df_orig=pd.read_csv('C:/Users/madhu/Desktop/credit_risk_dataset.csv')
train_df = pd.concat([train_df, df_orig])
train_df['person_emp_length'].fillna(train_df['person_emp_length'].median(), inplace=True)
train_df['loan_int_rate'].fillna(train_df['loan_int_rate'].median(), inplace=True)

# Fill missing values in test_df
test_df['person_emp_length'].fillna(test_df['person_emp_length'].median(), inplace=True)
test_df['loan_int_rate'].fillna(test_df['loan_int_rate'].median(), inplace=True)
train_df.describe()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,58645.000000,91226.000000,9.122600e+04,91226.000000,91226.000000,91226.000000,91226.000000,91226.000000,91226.000000
mean,29322.000000,27.616480,6.477071e+04,4.724936,9350.348607,10.794990,0.163154,5.810219,0.169447
std,16929.497605,6.148115,4.793696e+04,4.005924,5848.605423,3.055578,0.097491,4.038412,0.375149
min,0.000000,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.000000
25%,14661.000000,23.000000,4.000000e+04,2.000000,5000.000000,7.880000,0.090000,3.000000,0.000000
50%,29322.000000,26.000000,5.700000e+04,4.000000,8000.000000,10.950000,0.140000,4.000000,0.000000
75%,43983.000000,30.000000,7.800000e+04,7.000000,12000.000000,12.990000,0.220000,8.000000,0.000000
max,58644.000000,144.000000,6.000000e+06,123.000000,35000.000000,23.220000,0.830000,30.000000,1.000000


In [42]:
# columns_to_check = ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']

# # Loop through each specified column to check for outliers and filter directly
# for column in columns_to_check:
#     Q1_train = train_df[column].quantile(0.25)
#     Q3_train = train_df[column].quantile(0.75)
#     IQR_train = Q3_train - Q1_train

#     # Calculate lower and upper bounds for train_df
#     lower_bound_train = Q1_train - 1.5 * IQR_train
#     upper_bound_train = Q3_train + 1.5 * IQR_train

#     # Remove rows with outliers directly from train_df
#     train_df = train_df[(train_df[column] >= lower_bound_train) & (train_df[column] <= upper_bound_train)]

#     # Now repeat the process for test_df
#     Q1_test = test_df[column].quantile(0.25)
#     Q3_test = test_df[column].quantile(0.75)
#     IQR_test = Q3_test - Q1_test

#     # Calculate lower and upper bounds for test_df
#     lower_bound_test = Q1_test - 1.5 * IQR_test
#     upper_bound_test = Q3_test + 1.5 * IQR_test

#     # Remove rows with outliers directly from test_df
#     test_df = test_df[(test_df[column] >= lower_bound_test) & (test_df[column] <= upper_bound_test)]

In [43]:
import pandas as pd

# Function to choose best fit for replacing outliers
def best_fit_for_outliers(series):
    mean_value = series.mean()
    median_value = series.median()
    skewness = series.skew()
    
    # Calculate IQR
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    num_outliers = outliers.count()

    # Determine replacement value based on skewness and number of outliers
    if skewness > 1 or skewness < -1:
        replacement = median_value  # Data is skewed, prefer median
    elif num_outliers > 0 and abs(mean_value - median_value) > 1e-5:
        replacement = median_value  # Outliers present and mean differs from median
    else:
        replacement = mean_value  # Data is symmetric or no significant outliers

    return replacement

# Define the columns to check for outliers
columns_to_check = ['person_age', 'person_income', 'person_emp_length', 
                    'loan_amnt', 'loan_int_rate', 'loan_percent_income', 
                    'cb_person_cred_hist_length']

# Loop through each specified column to check for outliers
for column in columns_to_check:
    # Replace outliers in train_df with the chosen replacement value
    train_replacement = best_fit_for_outliers(train_df[column])
    train_df[column] = train_df[column].mask((train_df[column] < (train_df[column].quantile(0.25) - 1.5 * (train_df[column].quantile(0.75) - train_df[column].quantile(0.25)))) | 
                                              (train_df[column] > (train_df[column].quantile(0.75) + 1.5 * (train_df[column].quantile(0.75) - train_df[column].quantile(0.25)))), 
                                              train_replacement)

    # Replace outliers in test_df with the chosen replacement value
    test_replacement = best_fit_for_outliers(test_df[column])
    test_df[column] = test_df[column].mask((test_df[column] < (test_df[column].quantile(0.25) - 1.5 * (test_df[column].quantile(0.75) - test_df[column].quantile(0.25)))) | 
                                             (test_df[column] > (test_df[column].quantile(0.75) + 1.5 * (test_df[column].quantile(0.75) - test_df[column].quantile(0.25)))), 
                                             test_replacement)


In [44]:
train_df = pd.get_dummies(train_df, drop_first=True)
test_df = pd.get_dummies(test_df, drop_first=True)
X = train_df.drop([ 'loan_status'], axis=1)
y = train_df['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
import optuna
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Assume X, y are your feature matrix and target vector
# Split data into train and test sets

# Define objective function for Optuna optimization
def objective(trial):
    # Suggest values for hyperparameters
    param = {
        'objective': 'binary:logistic',  
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'random_state': 42  # for reproducibility
    }

    # Train model with suggested parameters
    model = xgb.XGBClassifier(**param)

    # Fit the model
    model.fit(X_train, y_train)

    # Predict and calculate AUC on the test set
    preds = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, preds)
    
    return auc

# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get the best parameters from the optimization
best_params = study.best_params
print("Best Parameters:", best_params)

# Initialize the XGBoost model with the best parameters
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    **best_params,  # Unpack the best parameters found by Optuna
    random_state=42  # for reproducibility
)

# Fit the model to the training data with the best parameters
xgb_model.fit(X_train, y_train)


[I 2024-10-30 20:01:40,776] A new study created in memory with name: no-name-202e78af-6718-40a4-8f30-848df53f0b43
[I 2024-10-30 20:01:41,106] Trial 0 finished with value: 0.907350165487898 and parameters: {'learning_rate': 0.013327585563336236, 'max_depth': 5, 'n_estimators': 86, 'subsample': 0.8156589839347861, 'colsample_bytree': 0.8411579569900336, 'gamma': 4.8448391039444685, 'min_child_weight': 3}. Best is trial 0 with value: 0.907350165487898.
[I 2024-10-30 20:01:41,469] Trial 1 finished with value: 0.9500467120337966 and parameters: {'learning_rate': 0.10744914285589141, 'max_depth': 9, 'n_estimators': 68, 'subsample': 0.8078588768683065, 'colsample_bytree': 0.9570062856274921, 'gamma': 3.2031704281475895, 'min_child_weight': 4}. Best is trial 1 with value: 0.9500467120337966.
[I 2024-10-30 20:01:41,770] Trial 2 finished with value: 0.9295399781650909 and parameters: {'learning_rate': 0.11641903982023169, 'max_depth': 3, 'n_estimators': 94, 'subsample': 0.8872270629177517, 'cols

Best Parameters: {'learning_rate': 0.11438435193025745, 'max_depth': 10, 'n_estimators': 112, 'subsample': 0.8770284051255107, 'colsample_bytree': 0.8234822595969286, 'gamma': 1.9483449316821617, 'min_child_weight': 2}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8234822595969286, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=1.9483449316821617,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.11438435193025745,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=112, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [47]:

y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Calculate accuracy and AUC
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test ROC AUC: {roc_auc:.4f}")

Test Accuracy: 0.9442
Test ROC AUC: 0.9542
